## SITCOMTN_101 Fan coil units (FCU) induced vibrations in M1M3

This notebook analyses data taken on December 7th, 2023 to check the vibrations induced by the Fan Cooling Units (FCU) on M1M3.
During the test, the fan speed varied from 0% up to 100% of the maximum speed (2550 rpm according to Petr Kubanek).
The description of the FCU system is available in [SPIE 77331E](https://www.spiedigitallibrary.org/conference-proceedings-of-spie/7733/77331E/LSST-primarytertiary-mirror-thermal-control-system/10.1117/12.857438.short)

The notebook has originally been developped by Peter Ferguson.
In this new vesion we choose to measure the PSD acceleration and displacement independently for the 3 axes (x, y, z) of the accelerometers instead of considering the norm of the acceleration vector. 

We also check that during each measurement the mirror was raised and stable.

In [ ]:
import sys, time, os, asyncio
import scipy.stats as stats
from scipy.signal import find_peaks
from scipy import signal
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
from matplotlib.ticker import FormatStrFormatter
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from matplotlib import ticker
from datetime import datetime

from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient, clipDataToEvent, calcNextDay

from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState, TMAEvent
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient
from lsst.ts.xml.enums import MTM1M3

%matplotlib inline
%load_ext autoreload
%autoreload 
%config Completer.use_jedi = False

In [ ]:
# Utility functions
key_m1m3_dict={'1 X': 'm1m3_x_1', 
            '1 Y': 'm1m3_y_1', 
            '1 Z': 'm1m3_z_1', 
            '2 X': 'm1m3_x_2', 
            '2 Y': 'm1m3_z_2', # note these two have been 
            '2 Z': 'm1m3_y_2', # switched pending SUMMIT-7911
            '3 X': 'm1m3_x_3', 
            '3 Y': 'm1m3_y_3', 
            '3 Z': 'm1m3_z_3'
            }
key_m2_dict={'1 X': 'm2_x_1', 
            '1 Y': 'm2_y_1', 
            '1 Z': 'm2_z_1', 
            '2 X': 'm2_x_2', 
            '2 Y': 'm2_z_2', 
            '2 Z': 'm2_y_2', 
            '3 X': 'm2_x_3', 
            '3 Y': 'm2_z_3', 
            '3 Z': 'm2_y_3', 
            '4 X': 'm2_x_4', 
            '4 Y': 'm2_y_4', 
            '4 Z': 'm2_z_4', 
            '5 X': 'm2_x_5', 
            '5 Y': 'm2_z_5', 
            '5 Z': 'm2_y_5',
            '6 X': 'm2_x_6', 
            '6 Y': 'm2_z_6', 
            '6 Z': 'm2_y_6', 
            }
def vms_data_to_pandas(filename, vms_type, begin_time=None, end_time=None):
    """
    Converts VMS data in the given HDF5 file to a Pandas DataFrame.

    Args:
    filename: Path to the HDF5 file containing the VMS data.
    vms_type: The type of VMS data in the file. Must be "m1m3", "m2", or
      "rotator".
    begin_time: The start time of the data to include in the DataFrame. If None,
      all data will be included.
    end_time: The end time of the data to include in the DataFrame. If None, all
      data will be included.

    Returns:
    A Pandas DataFrame containing the VMS data.
    """
    if vms_type == "m1m3":
        key_dict=key_m1m3_dict
    elif vms_type=="m2":
        key_dict=key_m2_dict
    elif vms_type=="rotator":
        raise NotImplementedError
    else:
        raise ValueError('vms_type must be m1m3,m2, or rotator')

    f = h5py.File(filename, 'r')
    times = f['timestamp'][::1]
    dkeys = 'XYZ'
   
    data_dict = {}
    if (begin_time is not None) & (end_time is not None): 
        sel = (times > begin_time) & (times < end_time)
    else: 
        sel = np.ones(times.size).astype(bool)
    data_dict['times'] = times[sel]  
    for key in key_dict.keys():
        # multiply values stored in hdf5 files by 2 in order to convert the acceleration values to mg
        data_dict[key_dict[key]] = f[key][::1][sel]*2.
    data_frame = pd.DataFrame(data_dict)
    for j in np.arange(int(len(key_dict)/3)) +1:
        data_frame[f"total_{j}"] = np.linalg.norm(
            data_frame[[f"{vms_type}_{i}_{j}" for i in ["x","y","z"]]].values, axis=1
        )
    
    
    return data_frame

def get_freq_psd(vals, timestep):

    """
    Calculates the frequency power spectrum of a signal.

    Args:
        vals (np.array): The signal values.
        timestep (float): The time step between samples.

    Returns:
        tuple: The frequencies and power spectral density.
    """

    # Remove the mean from the signal.

    meanval = np.mean(vals)
    signal = vals - meanval

    # Calculate the length of the signal.

    N = len(signal)

    # Calculate the power spectral density.

    psd = np.abs(np.fft.rfft(np.array(signal) * 1)) ** 2

    # Calculate the frequencies.

    frequencies = np.fft.rfftfreq(N, timestep)
    return (frequencies, psd)
    
def get_psd_and_dsd_for_vms(vals, timestep, min_freq=1, g=False):
    """
    Calculate the PSD and DSD from VMS data, and total displacement from DSD.

    Parameters:
    - vals (array-like): VMs in m/s^2 or milli-g if 'g' is True.
    - timestep (float): Time step between measurements in seconds.
    - min_freq (float, optional): Minimum frequency for calculations.
                                  Default is 1 Hz.
    - g (bool, optional): True if 'vals' are in milli-g units.
                          Default is False.

    Returns:
    - psds_df (DataFrame): DataFrame with frequencies ('freq'), acceleration
                           PSD ('accel_psd'), displacement PSD ('disp_psd'),
                           and cumulative displacement PSD ('int_disp').
    - total_displacement (float): Total displacement from vibration data.

    Note:
    PSD adjusted to m/s^2/Hz. 'vals' converted to m/s^2 if 'g' is True.
    """
    if g:
        vals = 1e-3 * 9.8 * vals

    #freq, accel_psd = signal.periodogram(vals, 1/timestep)
    #print("using scipy")
    freq, accel_psd = get_freq_psd(vals, timestep)

    sel = (freq > min_freq)
    freq = freq[sel]
    accel_psd = accel_psd[sel]

    accel_psd = accel_psd * np.mean(np.diff(freq))

    disp_psd_sq = accel_psd / ((2 * np.pi * freq) ** 4)

    int_displace_psd = np.sqrt(np.cumsum(disp_psd_sq))
    total_displacement = np.sqrt(np.sum(disp_psd_sq))

    psds_df = pd.DataFrame({"freq": freq, "accel_psd": accel_psd, 
                            "disp_psd": np.sqrt(disp_psd_sq),
                            "int_disp": int_displace_psd,
                           })
    
    return psds_df, total_displacement

def get_data(begin_time, end_time, client_usdf, client_idf=None):
    table_dict = {
        "m1m3_hp_actuator": "lsst.sal.MTM1M3.hardpointActuatorData",
        # "m1m3_hp_monitor": "lsst.sal.MTM1M3.hardpointMonitorData",
        # "mtmount_elevation": "lsst.sal.MTMount.elevation",
        "m1m3_accel": "lsst.sal.MTM1M3.accelerometerData",
        "m1m3_ims": "lsst.sal.MTM1M3.imsData"
    }
    cols_dict = {
        "m1m3_hp_actuator": ["*"],
        "m1m3_accel": ["*"],
        "m1m3_ims": ["*"]
    }
    query_dict = {}
    for key in table_dict.keys():
        query_dict[key] = []
        if (end_time < datetime(2023, 11, 28)) & (client_idf is not None):
            client = client_idf
        else:
            client = client_usdf
            
        query = getEfdData(
                    client=client,
                    topic=table_dict[key],
                    columns=cols_dict[key],
                    begin=begin_time,
                    end=end_time,
                    prePadding=0,
                    postPadding=0,
                )
        query = add_timestamp(query)

        query_dict[key].append(query)
        
    for key in table_dict.keys():
        query_dict[key] = pd.concat(query_dict[key])
        
    return query_dict
    
def add_timestamp(data):
    """
    Adds a correct timestamp column in UTC format to the provided data if
    not present.

    Parameters
    ----------
    data : DataFrame
        The data to which the timestamp will be added.

    Returns
    -------
    DataFrame
        The data with the added 'snd_timestamp_utc' column.
    """
    if "snd_timestamp_utc" not in data.columns:
        data["snd_timestamp_utc"] = Time(
            data["private_sndStamp"], format="unix_tai"
        ).unix
    return data

In [ ]:
# hdf5 file containing the accelerometer data
vms_m1m3_filename="/sdf/data/rubin/shared/mtm1m3_test_files/vms_data/2023/12/M1M3-2023-12-07T00:00.hdf"

In [ ]:
# Create a directory to save plots

plot_dir = "./plots"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

# Analyse data corresponding to the various FCU speeds
1. Read in CSV of timestamps, file `fcu_timestamps-corrected.csv` is attached to ticket sitcom 1131 (an embeded version is also provided for convenience)
2. Check mirror position and stability
3. Plot of all sensors acceleration PSD for every axis and for all fan states
4. Plot of all sensors displacement PSD for every axis and for all fan states

In [ ]:
# For convenience and in order for this notebook to be able to run without copying data from somewhere else
# the content of the original csv file is loaded here as a dictionnary
# The fcu_state corresponds to the fan speed expressed as a percentage of the maximum speed
# The 2 other columns contain the timestamps corresponding to the beginning and end of a given measurement period with the fans running at a constant speed.

fcu_timestamps = {"fcu_state":[0, 100, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                  " begin":["2023-12-07 14:20:00.00", "2023-12-07 14:33:00.00", "2023-12-07 14:55:30.00", "2023-12-07 15:00:05.00", "2023-12-07 15:05:05.00",
                           "2023-12-07 15:12:30.00", "2023-12-07 15:16:15.00", "2023-12-07 15:21:40.00", "2023-12-07 15:26:05.00", "2023-12-07 15:31:05.00",
                           "2023-12-07 17:48:55.00", "2023-12-07 17:51:30.00"],
                  " end":["2023-12-07 14:30:00.00", "2023-12-07 14:45:00.00", "2023-12-07 14:59:55.00", "2023-12-07 15:04:00.00", "2023-12-07 15:10:00.00",
                         "2023-12-07 15:14:30.00", "2023-12-07 15:20:00.00", "2023-12-07 15:25:00.00", "2023-12-07 15:30:00.00", "2023-12-07 15:40:00.00",
                         "2023-12-07 17:50:10.00", "2023-12-07 17:53:00.00"]}

In [ ]:
file_path = "fcu_timestamps-corrected.csv"  # Path to file with timestamps and FCU speeds
if os.path.exists(file_path):
    print(f"Reading time stamps and FCU speeds from file: {file_path}")
    fcu_times = pd.read_csv(file_path)
else:
    print("Reading time stamps and FCU speeds from dictionnary")
    fcu_times = pd.DataFrame(fcu_timestamps)
    
fcu_times=fcu_times.sort_values("fcu_state").reset_index(drop=True)

# Create table with fan speed and data taking periods for technote 

In [ ]:
header ="""|FCU state| Start Time             | End Time               |
+=========+========================+========================+"""
row_str="+---------+------------------------+------------------------+"


In [ ]:
print(row_str)
print(header)
for i in range(len(fcu_times)):
    percentv,begin, end = fcu_times.iloc[i,:]
    print(f"| {percentv:7d} | {begin} | {end} |")
    print(row_str)

## Check that the TMA doesn't move during the vibration measurements

In [ ]:
client = EfdClient("usdf_efd")

In [ ]:
# Check whether the TMA stayed at the same azimuth and elevation during the vibration measurements
fig, ax = plt.subplots(3, 4,dpi=125,figsize=(15,8))
for i in range(len(fcu_times)):
    percentv,begin, end = fcu_times.iloc[i,:]
    begin=Time(begin)
    end=Time(end)
    # Get TMTMount Elevation data
    df_ele = getEfdData(client, "lsst.sal.MTMount.elevation", begin=begin, end=end)
    # Get TMTMount Azimuth data
    df_azi = getEfdData(client, "lsst.sal.MTMount.azimuth", begin=begin, end=end)

    l = int(i/4)
    c = i%4
    _ = ax[l,c].plot(df_ele.index, df_ele["actualPosition"], label="elev")
    _ = ax[l,c].plot(df_azi.index, df_azi["actualPosition"], label="azi")
    ax[l][c].set(ylabel="TMA Angle (deg)", xlabel="Time", title=f"Speed {percentv}%")
    ax[l][c].legend()
   
    ax[l][c].xaxis.set_major_formatter(DateFormatter("%H-%M-%S"))
    plt.setp(ax[l][c].get_xticklabels(), rotation = 45)

fig.tight_layout()
fig.savefig(plot_dir+"/TMA_position.png")

## Spikes
We see that the EFD MTMount.elevation and MTMount.azimuth topics exhibit several spikes where the TMA seems to jump from 89.95 deg to 0 deg 
in elevation and from -25.95 deg to 0 deg in azimuth. 

These spikes look suspiciously narrow and incompatible with an actual move of the TMA

Let's zoom in on one of these spikes

In [ ]:
# Check whether the TMA is stable and at the same position during the vibration measurements
i = 10
fig, ax = plt.subplots(1,dpi=100,figsize=(8,5))
percentv,begin, end = fcu_times.iloc[i,:]
begin=Time(begin)
end=Time(end)
print(begin, end)
begin = Time("2023-12-07 14:35:18.000")
end = Time("2023-12-07 14:35:22.000")
# Get TMTMount Elevation data
df_ele = getEfdData(client, "lsst.sal.MTMount.elevation", begin=begin, end=end)
# Get TMTMount Azimuth data
df_azi = getEfdData(client, "lsst.sal.MTMount.azimuth", begin=begin, end=end)

_ = ax.plot(df_ele.index, df_ele["actualPosition"], label="elev")
_ = ax.set(ylabel="TMA Angle (deg)", xlabel="Time", title=f"Speed {percentv}%")

fig.savefig(plot_dir+"/TMA_spike.png")

## Conclusion on the spikes and TMA position during vibration measurements

We clearly see that the spikes reported by the EFD cannot be associated to actual move of the TMA

We conclude that the TMA orientation in both azimuth and elevation was stable during the vibration measurements

## Check mirror position and stability during vibration measurements

In the following we check that :

 * the mirror is raised (MTM1M3.detailedStates == ACTIVEENGINEERING) and stable during each measurement period
 * the Hard Points are in standby state and stable during each measurement period

In [ ]:
# Retrieve the M1M3 detailed states during a time period including the FCU tests
begin_time=Time('2023-12-07 12:00:00.00')
end_time=Time('2023-12-07 19:00:00.00')
det_state = getEfdData(client, "lsst.sal.MTM1M3.logevent_detailedState", begin=begin_time, end=end_time)
det_state["detailedStateName"] = \
    det_state["detailedState"].map(lambda x: MTM1M3.DetailedStates(x).name)

# Retrieve M1M3 Hard Point states during a time period including the FCU tests
hp_state = getEfdData(client, "lsst.sal.MTM1M3.logevent_hardpointActuatorState", begin=begin_time, end=end_time)

# Get forceControllerState info to check whether the Force Balance systeme is on or not.
fc_state = getEfdData(client, "lsst.sal.MTM1M3.logevent_forceControllerState", begin=begin_time, end=end_time)

In [ ]:
header ="""|FCU state| Start Time              | End Time                | Detailed State    | Stable |
+=========+=========================+=========================+===================+========+"""
row_str="+---------+-------------------------+-------------------------+-------------------+--------+"

print(header)
for i in range(len(fcu_times)):
    percentv,begin, end = fcu_times.iloc[i,:]
    begin=Time(begin)
    end=Time(end)
    # Find time stamp in det_state preceding the beginning of the FCU run
    iloc_begin_st = det_state.index.get_indexer([begin], method='pad')[0]
    # Find time stamp in det_state preceding the end of the FCU run
    iloc_end_st = det_state.index.get_indexer([end], method='pad')[0]
    if iloc_begin_st == iloc_end_st :
        stable = True
    else:
        stable = False
    state = det_state["detailedStateName"].iloc[iloc_end_st]
    #print(percentv, begin, end, state, stable)
    print(f"| {percentv:7d} | {begin} | {end} | {state} | {stable}   |")
    print(row_str)

In [ ]:
header ="""|FCU state| Start Time              | End Time                | HP Standby    | HP Stable  |
+=========+=========================+=========================+===================+========+"""
row_str="+---------+-------------------------+-------------------------+-------------------+--------+"

print(header)
for i in range(len(fcu_times)):
    percentv,begin, end = fcu_times.iloc[i,:]
    begin=Time(begin)
    end=Time(end)
    # Find time stamp in det_state preceding the beginning of the FCU run
    iloc_begin = hp_state.index.get_indexer([begin], method='pad')[0]
    # Find time stamp in det_state preceding the end of the FCU run
    iloc_end = hp_state.index.get_indexer([end], method='pad')[0]
    if iloc_begin == iloc_end :
        hp_stable = True
    else:
        hp_stable = False
    sub_hp_state = hp_state.iloc[iloc_end] 
    # Check whether the 6 HP are in STANDBY mode
    hp_standby = (sub_hp_state[[f"motionState{j}" for j in range(0, 6)]] == 0).sum() == 6
    #print(percentv, begin, end, state, stable)
    print(f"| {percentv:7d} | {begin} | {end} | {hp_standby}          | {hp_stable}       |")
    print(row_str)

## Check whether the Force Balance System is on or not during FCU test

In [ ]:
header ="""|FCU state| Start Time              | End Time                | BF Applied     | BF Stable |
+=========+=========================+=========================+================+===========+"""
row_str="+---------+-------------------------+-------------------------+----------------+-----------+"

print(header)
for i in range(len(fcu_times)):
    percentv,begin, end = fcu_times.iloc[i,:]
    begin=Time(begin)
    end=Time(end)
    # Find time stamp in det_state preceding the beginning of the FCU run
    iloc_begin = fc_state.index.get_indexer([begin], method='pad')[0]
    # Find time stamp in det_state preceding the end of the FCU run
    iloc_end = fc_state.index.get_indexer([end], method='pad')[0]
    if iloc_begin == iloc_end :
        fc_stable = True
    else:
        fc_stable = False
    sub_fc_state = fc_state.iloc[iloc_end] 
    # Check whether the 6 HP are in STANDBY mode
    fc_standby = sub_fc_state["balanceForcesApplied"]
    #print(percentv, begin, end, state, stable)
    print(f"| {percentv:7d} | {begin} | {end} | {fc_standby}          | {fc_stable}      |")
    print(row_str)

## Conclusion on the mirror status during the FCU tests

We conclude that during the FCU tests, the mirror was raised and stable with Hard Points in standby mode.
The Force Balance System was turned off for the whole duration of the tests.

In [ ]:
# Read data from hdf file into pandas
begin_time=Time(fcu_times[' begin'].min())
end_time=Time(fcu_times[' end'].max())
vms_m1m3_data=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_time.unix, end_time=end_time.unix)

## Check sampling rate

As we are going to estimate a PSD, it is important to check that the sampling rate is constant.
It is supposed to be equal to 200 Hz

In [ ]:
subdat_sel = (vms_m1m3_data["times"] > begin.unix) & (vms_m1m3_data["times"] < end.unix) 
subdat=vms_m1m3_data.loc[subdat_sel,:]
delta_t = np.diff(subdat["times"])

fig, ax = plt.subplots(1,2,dpi=100,figsize=(15,7))
_ = ax[0].hist(delta_t, bins=50)
ax[0].set_xlabel("delta t (s)")
ax[0].set_yscale("log")

sample = np.arange(len(delta_t))
_ = ax[1].plot(sample, delta_t)
ax[1].set_xlabel("Sample nbr.")
ax[1].set_ylabel("delta t (s)")

fig.savefig(plot_dir+"/sampling.png")

## Conclusion on the sampling rate

From the previous plots we see that the sampling rate is equal to ~200 Hz for most of the data samples but for a fraction of the samples the time difference between 2 consecutive 
samples is equal to ~0.00444 instead of 0,0050. On the right plot where these differences are plotted with respect to sampling number, one can see that these occurences are periodic. If real, this could potentially bias the PSD measurements, nevertheless from a discussion with Petr Kubanek it seems that the sampling rate is set to 200 Hz by the hardware and that the observed behavior is probably due to wrong timestamp.

This issue as been logged in ticket DM-43219

# Plot VMS data for an initial time range 

Zoom on a short time frame (10s) and apply some filtering in order to exhibit a possible low frequency behavior.

In [ ]:
table="lsst.sal.MTM1M3.accelerometerData"
# Fan at 100% speed
begin_time=Time('2023-12-07 14:40:00.00')
end_time=Time('2023-12-07 14:40:10.00')
# Fan at 0% speed
#begin_time=Time('2023-12-07 14:25:00.00')
#end_time=Time('2023-12-07 14:25:10.00')

In [ ]:
vms_m1m3_data=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_time.unix, end_time=end_time.unix)

In [ ]:
fig, axs = plt.subplots(3,1, sharex=True, figsize=(12,5), dpi=175)
accel_smooth=50
vms_times=Time(vms_m1m3_data["times"], format="unix").datetime
for i in range(3):
    ax=axs[i]
    ax.plot(vms_times, vms_m1m3_data[f"total_{i+1}"].rolling(50).mean(), label=i+1)
    
    ax.set_yticks(np.arange(0.001,0.002,0.001))
plt.subplots_adjust(hspace=0)
axs[1].set_ylabel("VMS total acceleration [m/s²]\nChannel 2")
axs[0].set_ylabel("Channel 1")
axs[2].set_ylabel("Channel 3")
ax.set_xlabel("Time")
fig.suptitle(f"M1M3 FCU #1\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n Total Vibration", y=0.99)
plt.savefig(f"{plot_dir}/vms_telemetry_zoom_1.png")

In [ ]:
# Plot total VMS acceleration for all 3 sensors for the first period at 100% speed and compare to the period at 0% speed

percentv,begin_off, end_off = fcu_times.iloc[0,:]
percentv,begin_on, end_on = fcu_times.iloc[10,:]
begin_off = Time(begin_off)
begin_on = Time(begin_on)
end_off = Time(end_off)
end_on = Time(end_on)

vms_m1m3_data_on=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_on.unix, end_time=end_on.unix)
#vms_m1m3_data_on["total_1"][vms_m1m3_data_on["total_1"] > 0.0035] = np.mean(vms_m1m3_data_on["total_1"])
vms_m1m3_data_off=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_off.unix, end_time=end_off.unix)

In [ ]:
fig, axs = plt.subplots(3,1, sharex=True, figsize=(12,5), dpi=175)

vms_times_on=Time(vms_m1m3_data_on["times"], format="unix").datetime
vms_times_off=Time(vms_m1m3_data_off["times"], format="unix").datetime
for i in range(3):
    ax=axs[i]
    ax.plot(vms_times_on, vms_m1m3_data_on[f"total_{i+1}"], label=f"FCUs on")
    ax.plot(vms_times_off, vms_m1m3_data_off[f"total_{i+1}"], label=f"FCUs off")
    
    ax.set_yticks(np.arange(0.000,0.009,0.002))
    ax.set_ylim(-0.001,0.009)
plt.subplots_adjust(hspace=0)
axs[0].legend()
axs[1].set_ylabel("VMS total acceleration [m/s²]\nSensor 2 2")
axs[0].set_ylabel("Sensor 1")
axs[2].set_ylabel("Sensor 3")
ax.set_xlabel("Time")
fig.suptitle(f"M1M3 FCU #1\n{begin_off.iso[:10]} {begin_off.iso[11:19]}-{end_on.iso[11:19]}\n Total Vibration", y=0.99)
plt.savefig(f"{plot_dir}/vms_telemetry_1.png")

In [ ]:
# Plot total VMS acceleration for all 3 sensors for the second period at 100% speed

percentv,begin_on, end_on = fcu_times.iloc[11,:]
begin_on = Time(begin_on)
end_on = Time(end_on)

vms_m1m3_data_on=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_on.unix, end_time=end_on.unix)
#vms_m1m3_data_on["total_1"][vms_m1m3_data_on["total_1"] > 0.0035] = np.mean(vms_m1m3_data_on["total_1"])
vms_m1m3_data_off=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_off.unix, end_time=end_off.unix)

In [ ]:
fig, axs = plt.subplots(3,1, sharex=True, figsize=(12,5), dpi=175)

vms_times_on=Time(vms_m1m3_data_on["times"], format="unix").datetime
vms_times_off=Time(vms_m1m3_data_off["times"], format="unix").datetime
for i in range(3):
    ax=axs[i]
    ax.plot(vms_times_on, vms_m1m3_data_on[f"total_{i+1}"], label=f"FCUs on")
#    ax.plot(vms_times_off, vms_m1m3_data_off[f"total_{i+1}"], label=f"FCUs off")
    
    ax.set_yticks(np.arange(0.000,0.009,0.002))
    ax.set_ylim(-0.001,0.009)
plt.subplots_adjust(hspace=0)
axs[0].legend()
axs[1].set_ylabel("VMS total acceleration [m/s²]\nSensor 2")
axs[0].set_ylabel("Sensor 1")
axs[2].set_ylabel("Sensor 3")
ax.set_xlabel("Time")
fig.suptitle(f"M1M3 FCU\n{begin_on.iso[:10]} {begin_on.iso[11:19]}-{end_on.iso[11:19]}\n Total Vibration", y=0.99)
plt.savefig(f"{plot_dir}/vms_telemetry_2.png")

## PSD Analysis

In [ ]:
# Read data from hdf file into pandas
begin_time=Time(fcu_times[' begin'].min())
end_time=Time(fcu_times[' end'].max())
vms_m1m3_data=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_time.unix, end_time=end_time.unix)

In [ ]:
# PSD (acceleration) for a given sensor (1, 2, or 3), a given axis (x, y, z) and for each speed value (0% - 100%) 

fig, ax = plt.subplots(3, 4,dpi=125,figsize=(10,6))

sensor = 2 # Sensor number
axis = "z"

for i in range(12):
    percentv,begin, end = fcu_times.iloc[i,:]
    begin=Time(begin)
    end=Time(end)
    subdat_sel = (vms_m1m3_data["times"] > begin.unix) & (vms_m1m3_data["times"] < end.unix) 
    subdat=vms_m1m3_data.loc[subdat_sel,:]
    
    key= f"m1m3_{axis}_{sensor}"
    psds_df, disp =get_psd_and_dsd_for_vms(subdat[key], 
                                                np.mean(np.diff(subdat["times"])),
                                                g=True, min_freq=1)
    x = psds_df['freq']
    y = psds_df['accel_psd']
    l = int(i/4)
    c = i%4
    ax[l][c].plot(x,y, label=f"{percentv}%\n", zorder=9, lw=1)
    ax[l][c].set_xticks(np.arange(0,110,20))
    ax[l][c].set(ylabel="psd (m/$s^2$)/hz", xlabel="Frequency [Hz]", title=f"Speed {percentv}% - axis {axis}")
    yticks = ticker.MaxNLocator(4)
    ax[l][c].yaxis.set_major_locator(yticks)
    
fig.suptitle(f"Sensor: {sensor}")
fig.tight_layout()
plt.savefig(f"{plot_dir}/psd_sensor_{sensor}_axis_{axis}.png")

In [ ]:
# Plot the PSD (aceleration) for each sensor, each axis and one given fan speed

cut_40 = False # If True then mask data around 40 Hz

i = 11 # Measurement number => check table of measurements above
fig, ax = plt.subplots(3, 3,dpi=125,figsize=(10,7))
for c in range(3):
    key=f"total_{c+1}"

    percentv,begin, end = fcu_times.iloc[i,:]
    begin=Time(begin)
    end=Time(end)
    subdat_sel = (vms_m1m3_data["times"] > begin.unix) & (vms_m1m3_data["times"] < end.unix) 
    subdat=vms_m1m3_data.loc[subdat_sel,:]
    for j, axis in enumerate("xyz"):
        key= f"m1m3_{axis}_{c+1}"
        psds_df, disp =get_psd_and_dsd_for_vms(subdat[key], 
                                                    np.mean(np.diff(subdat["times"])),
                                                    g=True, min_freq=1)
        if cut_40:
            cut = (psds_df['freq'] < 35) | (psds_df['freq'] > 45)
        else:
            cut = np.full(len(psds_df['freq']), True)
            
        x = psds_df['freq'][cut]
        y = psds_df['accel_psd'][cut]
        ax[c][j].plot(x,y, label=f"{percentv}%\n", zorder=9, lw=1)
        ax[c][j].set_xticks(np.arange(0,110,10))
        ax[c][j].set(ylabel="psd (m/$s^2$)/hz", xlabel="Frequency [Hz]", title=f"S{c+1} - axis {axis}")
        yticks = ticker.MaxNLocator(4)
        ax[c][j].yaxis.set_major_locator(yticks)
fig.suptitle(f"FCU speed: {percentv}%")
fig.tight_layout()
plt.savefig(f"{plot_dir}/psd_speed_{percentv}.png")

In [ ]:
# Total (cumulative) displacement for a given sensor, a given axis and all the speed values

fig, ax = plt.subplots(3, 4,dpi=125,figsize=(10,6))

sensor = 3 # Sensor number
axis = "z"

for i in range(12):
    percentv,begin, end = fcu_times.iloc[i,:]
    begin=Time(begin)
    end=Time(end)
    subdat_sel = (vms_m1m3_data["times"] > begin.unix) & (vms_m1m3_data["times"] < end.unix) 
    subdat=vms_m1m3_data.loc[subdat_sel,:]
    
    #key = f"total_{sensor}"
    key= f"m1m3_{axis}_{sensor}"
    psds_df, tot_disp =get_psd_and_dsd_for_vms(subdat[key], 
                                                np.mean(np.diff(subdat["times"])),
                                                g=True, min_freq=1)
    x = psds_df['freq']
    y = psds_df['int_disp']*1.e6
    l = int(i/4)
    c = i%4
    ax[l][c].plot(x,y, label=f"{percentv}%\n", zorder=9, lw=1)
    ax[l][c].set_xticks(np.arange(0,110,20))
    ax[l][c].set(ylabel="$\Sigma$ disp [$\mu m$]", xlabel="Frequency [Hz]", title=f"Speed {percentv}%")
    ax[l,c].text(0.6, 0.8, "{:#.2g} $\mu m$".format(tot_disp*1.e6), transform=ax[l][c].transAxes)
    yticks = ticker.MaxNLocator(4)
    ax[l][c].yaxis.set_major_locator(yticks)
    
fig.suptitle(f"Sensor: {sensor} - {axis}  axis")
fig.tight_layout()
plt.savefig(f"{plot_dir}/psd_cumul_disp_sensor_{sensor}_axis_{axis}.png")

## Investigation using data from the EFD

In [ ]:
client_usdf = EfdClient("usdf_efd")

begin_time_on=Time('2023-12-07 14:33:00.00')
end_time_on=Time('2023-12-07 14:50:00.00')

begin_time_off=Time('2023-12-07 14:20:00.00')
end_time_off=Time('2023-12-07 14:30:00.00')

query_dict={}
query_dict["on"]=get_data(begin_time_on, end_time_on, client_usdf)
query_dict["off"]=get_data(begin_time_off, end_time_off, client_usdf)

In [ ]:
fig, axs = plt.subplots(6,1, sharex=True, figsize=(12,10), dpi=175)


for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_hp_actuator"]
    for ax_id, val in enumerate([f"measuredForce{i}" for i in range(6)]):
        ax=axs[ax_id]
        ax.plot(Time(data_df["snd_timestamp_utc"], format="unix").datetime, 
                data_df[val], label=f"FCUs {key}")
        ax.set_ylabel(f"HP_{ax_id}\nForce [N]")
        minv,maxv=ax.get_ylim()
        ax.set_yticks(np.arange(np.round(minv/100,0)*100,np.round(maxv/100,0)*100 + 50, 50))
        #ax.set_yticks(np.arange(np.round(minv,4),maxv, 0.0005))
        # if val == "z":
        #     ax.set(ylim=(meanv - 4 ,meanv + 20), yticks=np.arange(int(meanv-4), meanv + 20, 5))
        # else:
        #     ax.set(ylim=(meanv - 4 ,meanv + 4))
        # ax.set(ylim=(meanv ,meanv + 4))
        if ax_id == 0:
            ax.legend()

axs[0].set_title("Hardpoint Forces")
axs[5].set_xlabel("Time [s]")
plt.subplots_adjust(hspace=0)

In [ ]:
fig, axs = plt.subplots(6,1, sharex=True, figsize=(12,10), dpi=175)


for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_hp_actuator"]
    for ax_id, val in enumerate([f"measuredForce{i}" for i in range(6)]):
        ax=axs[ax_id]
        freq,accel_psd=get_freq_psd(data_df[val], np.mean(np.diff(data_df["snd_timestamp_utc"])))
        sel = (freq > 1)
        freq = freq[sel]
        accel_psd = accel_psd[sel]
        
        accel_psd = np.sqrt(accel_psd * np.mean(np.diff(freq)))
        ax.plot(freq, accel_psd, 
                label=f"FCUs {key}")
        ax.set_ylabel(f"HP_{ax_id}\nForce [N/Hz]")
        minv,maxv=ax.get_ylim()
        # ax.set_yticks(np.arange(np.round(minv/100,0)*100,np.round(maxv/100,0)*100 + 50, 50))
        #ax.set_yticks(np.arange(np.round(minv,4),maxv, 0.0005))
        # if val == "z":
        #     ax.set(ylim=(meanv - 4 ,meanv + 20), yticks=np.arange(int(meanv-4), meanv + 20, 5))
        # else:
        #     ax.set(ylim=(meanv - 4 ,meanv + 4))
        # ax.set(ylim=(meanv ,meanv + 4))
        if ax_id == 0:
            ax.legend()

axs[0].set_title("Hardpoint Forces")
axs[5].set_xlabel("Hz")
plt.subplots_adjust(hspace=0)

In [ ]:
axis_table = { 0: "1_x",
               1: "1_y",
               2: "2_x",
               3: "2_y",
               4: "3_x",
               5: "3_y", 
               6: "4_x",
               7: "4_y",
}
fig, axs = plt.subplots(8,1, sharex=True, figsize=(12,10/6*8), dpi=175)
for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_accel"]
    for ax_id, val in enumerate([f'accelerometer{i}' for i in range(8)]):
        ax=axs[ax_id]
        ax.plot(Time(data_df["snd_timestamp_utc"], format="unix").datetime, 
                data_df[val], label=f"FCUs {key}")
        ax.set_ylabel(f"accel {axis_table[ax_id]}\n[m/s^2]")
        minv,maxv=ax.get_ylim()
        #ax.set_yticks(np.arange(np.round(minv/100,0)*100,np.round(maxv/100,0)*100 + 50, 50))
        ax.set_yticks(np.linspace(np.round(minv,3),np.round(maxv,3), 4))
        # if val == "z":
        #     ax.set(ylim=(meanv - 4 ,meanv + 20), yticks=np.arange(int(meanv-4), meanv + 20, 5))
        # else:
        #     ax.set(ylim=(meanv - 4 ,meanv + 4))
        # ax.set(ylim=(meanv ,meanv + 4))
        if ax_id == 0:
            ax.legend()
axs[0].set_title("DC acceleromteters")
axs[7].set_xlabel("Time [s]")
plt.subplots_adjust()
plt.savefig(f"{plot_dir}/dc_accel_telemetry.png")

In [ ]:
axis_table = { 0: "1_x",
               1: "1_y",
               2: "2_x",
               3: "2_y",
               4: "3_x",
               5: "3_y", 
               6: "4_x",
               7: "4_y",
}
fig, axs = plt.subplots(8,1, sharex=True, figsize=(12,10/6*8), dpi=175)
for key in ["on","off"]:
    data_df=query_dict[key]["m1m3_accel"]
    for ax_id, val in enumerate([f'accelerometer{i}' for i in range(8)]):
        ax=axs[ax_id]
        freq,accel_psd=get_freq_psd(data_df[val], np.mean(np.diff(data_df["snd_timestamp_utc"])))
        sel = (freq > 1)
        freq = freq[sel]
        accel_psd = accel_psd[sel]
        
        accel_psd = np.sqrt(accel_psd * np.mean(np.diff(freq)))
        ax.plot(freq, accel_psd, 
                label=f"FCUs {key}")
        ax.set_ylabel(f"accel {axis_table[ax_id]}\n[(m/s^2)/Hz]")
        if ax_id == 0:
            ax.legend()
axs[0].set_title("DC accelerometers")
axs[7].set_xlabel("Time [s]")
plt.subplots_adjust(hspace=0)
plt.savefig("./plots/dc_accel_psd.png")